In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import sys
from __future__ import division
#sys.path.append("/home/gias/dev/opinion/opinion/python/opinion")
sys.path.append(r"C:\dev\opinion\opinion\python\opinion")
import os
import utils.fileutils as fileutils
import utils.nlputils as nlputils
import cPickle as pickle
import pandas as pd
import numpy as np
import utils.metrics as metrics

from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
stopWords = set(stopwords.words('english'))

import statsmodels.api as sm
import statsmodels.formula.api as smf

In [10]:
indir = r"C:\dev\opinion\papers\sentiplus"
datadir = os.path.join(indir, "dataset")

infileJira  = os.path.join(datadir, "DatasetLinJIRAWithToolOutputs.xls")
dfJira = fileutils.readExcel(infileJira, "Sheet1")
dfJira.head()

infileAppReviews  = os.path.join(datadir, "DatasetLinAppReviewsWithToolOutputs.xls")
dfAppReviews = fileutils.readExcel(infileAppReviews, "Sheet1")
dfAppReviews.head()

infileLinSO  = os.path.join(datadir, "DatasetLinStackOverflowWithToolOutputs.xls")
dfLinSO = fileutils.readExcel(infileLinSO, "Sheet1")
dfLinSO.head()

infileSenti4SD  = os.path.join(datadir, "DatasetSenti4SDWithToolOutputs.xls")
dfSenti4SD = fileutils.readExcel(infileSenti4SD, "Sheet1")
dfSenti4SD.head()

infileOpiner  = os.path.join(datadir, "DatasetUddinSO_Test_Con.xls")
dfOpiner = fileutils.readExcel(infileOpiner, "Sheet1")
dfOpiner.head()


,thread,id,sentence,Sentistrength-se,SentiSE,InitialLabel,Sentistrength,Lexicons,OpinerDSO,ManualLabel,tid,sid,OpinerDSOSenti,Senti4SD,Stanford,SentiCR,shannon,simpson,simpson_inv
0,15936368,1,JAXB Bindings File Sets @XmlElement type to St...,1 -1,o,o,o,NaN,o,o,15936368:1,1,o,o,o,o,2.484907,0.083333,12.000000
1,15936368,2,I'm trying to create an CODETERM1 that takes i...,1 -1,o,o,o,NaN,o,o,15936368:2,2,o,o,o,o,2.566599,0.085938,11.636364
2,15936368,3,The purpose is simlply to remove timezone data...,1 -1,o,o,o,NaN,o,o,15936368:3,3,o,o,o,o,2.540036,0.081633,12.250000
3,15936368,4,It looks like this: CODESNIPPET_JAVA1,1 -1,o,p,p,NaN,o,o,15936368:4,4,p,o,o,o,1.791759,0.166667,6.000000
4,15936368,5,This works fine for the following code: CODESN...,3 -1,p,p,p,works,p,p,15936368:5,5,p,p,p,o,2.197225,0.111111,9.000000


# Check for Correlation between Disagreements and Diversity Values

In [15]:
def createInputDataframeForOLS(df):
    data = []
    for index, row in df.iterrows():
        record = dict()
        sentence = row["sentence"]
        truth = row["ManualLabel"]
        sentiSE = row["SentiSE"]
        senti4SD = row["Senti4SD"]
        shannon = row["shannon"]
        disagreementBetweenTools = sentiSE != senti4SD
        disagreementsTruthSentiSE = truth != sentiSE
        disagreementsTruthSenti4SD = truth != senti4SD
        record["truth"] = truth
        record["shannon"] = shannon
        record["disagreement_sentise_senti4sd"] = disagreementBetweenTools
        record["disagreement_truth_sentise"] = disagreementsTruthSentiSE
        record["disagreement_truth_senti4sd"] = disagreementsTruthSenti4SD
        data.append(record)
    dfo = pd.DataFrame.from_dict(data)
    return dfo

In [16]:
dfo = createInputDataframeForOLS(dfJira)
dfo.head()

,disagreement_sentise_senti4sd,disagreement_truth_senti4sd,disagreement_truth_sentise,shannon,truth
0,False,False,False,1.747868,n
1,True,False,True,3.467994,n
2,True,True,False,2.564949,n
3,True,True,False,2.978508,n
4,True,True,False,3.262568,n


In [17]:
linreg = smf.ols(formula='shannon ~ disagreement_sentise_senti4sd + disagreement_truth_sentise + disagreement_truth_senti4sd', data=dfo).fit()

In [20]:
print linreg.summary()

                            OLS Regression Results                            
Dep. Variable:                shannon   R-squared:                       0.085
Model:                            OLS   Adj. R-squared:                  0.082
Method:                 Least Squares   F-statistic:                     28.51
Date:                Sat, 08 Sep 2018   Prob (F-statistic):           1.24e-17
Time:                        13:07:34   Log-Likelihood:                -938.90
No. Observations:                 926   AIC:                             1886.
Df Residuals:                     922   BIC:                             1905.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                                            coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------

In [21]:
dfo = createInputDataframeForOLS(dfAppReviews)
linreg = smf.ols(formula='shannon ~ disagreement_sentise_senti4sd + disagreement_truth_sentise + disagreement_truth_senti4sd', data=dfo).fit()
print linreg.summary()

                            OLS Regression Results                            
Dep. Variable:                shannon   R-squared:                       0.083
Model:                            OLS   Adj. R-squared:                  0.074
Method:                 Least Squares   F-statistic:                     10.12
Date:                Sat, 08 Sep 2018   Prob (F-statistic):           2.11e-06
Time:                        22:27:55   Log-Likelihood:                -393.38
No. Observations:                 341   AIC:                             794.8
Df Residuals:                     337   BIC:                             810.1
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                                            coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------

In [22]:
dfo = createInputDataframeForOLS(dfLinSO)
linreg = smf.ols(formula='shannon ~ disagreement_sentise_senti4sd + disagreement_truth_sentise + disagreement_truth_senti4sd', data=dfo).fit()
print linreg.summary()

                            OLS Regression Results                            
Dep. Variable:                shannon   R-squared:                       0.009
Model:                            OLS   Adj. R-squared:                  0.007
Method:                 Least Squares   F-statistic:                     4.612
Date:                Sat, 08 Sep 2018   Prob (F-statistic):            0.00323
Time:                        22:28:56   Log-Likelihood:                -1163.0
No. Observations:                1500   AIC:                             2334.
Df Residuals:                    1496   BIC:                             2355.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                                            coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------

In [24]:
dfo = createInputDataframeForOLS(dfOpiner)
linreg = smf.ols(formula='shannon ~ disagreement_sentise_senti4sd + disagreement_truth_sentise + disagreement_truth_senti4sd', data=dfo).fit()
print linreg.summary()

                            OLS Regression Results                            
Dep. Variable:                shannon   R-squared:                       0.012
Model:                            OLS   Adj. R-squared:                  0.011
Method:                 Least Squares   F-statistic:                     17.26
Date:                Sat, 08 Sep 2018   Prob (F-statistic):           3.90e-11
Time:                        22:30:10   Log-Likelihood:                -4680.0
No. Observations:                4228   AIC:                             9368.
Df Residuals:                    4224   BIC:                             9393.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                                            coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------

In [39]:
dfo = createInputDataframeForOLS(dfOpiner)
logit = smf.Logit(dfo['disagreement_truth_sentise'], dfo['shannon']).fit()
print logit.summary()

Optimization terminated successfully.
         Current function value: 0.678085
         Iterations 4
                               Logit Regression Results                               
Dep. Variable:     disagreement_truth_sentise   No. Observations:                 4228
Model:                                  Logit   Df Residuals:                     4227
Method:                                   MLE   Df Model:                            0
Date:                        Sat, 08 Sep 2018   Pseudo R-squ.:                -0.01067
Time:                                22:48:10   Log-Likelihood:                -2866.9
converged:                               True   LL-Null:                       -2836.7
                                                LLR p-value:                       nan
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
shannon       -0.1351      0.012    -11.181 

In [40]:
dfo = createInputDataframeForOLS(dfOpiner)
logit = smf.Logit(dfo['disagreement_truth_senti4sd'], dfo['shannon']).fit()
print logit.summary()

Optimization terminated successfully.
         Current function value: 0.693147
         Iterations 2
                                Logit Regression Results                               
Dep. Variable:     disagreement_truth_senti4sd   No. Observations:                 4228
Model:                                   Logit   Df Residuals:                     4227
Method:                                    MLE   Df Model:                            0
Date:                         Sat, 08 Sep 2018   Pseudo R-squ.:              -0.0002331
Time:                                 22:48:32   Log-Likelihood:                -2930.6
converged:                                True   LL-Null:                       -2929.9
                                                 LLR p-value:                       nan
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
shannon        0.0002      0.012    

In [41]:
dfo = createInputDataframeForOLS(dfOpiner)
logit = smf.Logit(dfo['disagreement_sentise_senti4sd'], dfo['shannon']).fit()
print logit.summary()

Optimization terminated successfully.
         Current function value: 0.602696
         Iterations 5
                                 Logit Regression Results                                
Dep. Variable:     disagreement_sentise_senti4sd   No. Observations:                 4228
Model:                                     Logit   Df Residuals:                     4227
Method:                                      MLE   Df Model:                            0
Date:                           Sat, 08 Sep 2018   Pseudo R-squ.:                -0.02180
Time:                                   22:49:02   Log-Likelihood:                -2548.2
converged:                                  True   LL-Null:                       -2493.8
                                                   LLR p-value:                       nan
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
shannon       -0.347

In [42]:
dfo = createInputDataframeForOLS(dfJira)
logit = smf.Logit(dfo['disagreement_truth_sentise'], dfo['shannon']).fit()
print logit.summary()

Optimization terminated successfully.
         Current function value: 0.582145
         Iterations 5
                               Logit Regression Results                               
Dep. Variable:     disagreement_truth_sentise   No. Observations:                  926
Model:                                  Logit   Df Residuals:                      925
Method:                                   MLE   Df Model:                            0
Date:                        Sat, 08 Sep 2018   Pseudo R-squ.:                 -0.1012
Time:                                22:54:29   Log-Likelihood:                -539.07
converged:                               True   LL-Null:                       -489.53
                                                LLR p-value:                       nan
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
shannon       -0.4636      0.035    -13.226 